# Perspectief 1: Verschillen in milieubeleid verklaren verschillen in landbouwuitstoot

De uitstoot van de landbouwsector verschilt sterk tussen landen. In dit perspectief onderzoeken we hoe beleidskeuzes zoals milieuregels en landbouwsubsidies bijdragen aan die verschillen. We gebruiken internationale data om te analyseren welke landen efficiënt produceren en welke landen vervuilende landbouw blijven stimuleren.


### Argument 1: Landen met strenger milieubeleid stoten gemiddeld minder CO₂ uit per ton landbouwproductie.



Landen verschillen sterk in hoe streng hun milieubeleid is. In sommige landen gelden duidelijke regels voor het gebruik van kunstmest, mestverwerking of methaanuitstoot. Andere landen leggen nauwelijks beperkingen op aan de landbouwsector. De aanname is dat streng milieubeleid boeren aanzet tot efficiëntere en schonere productie.

We gebruiken de Environmental Performance Index (EPI) als indicator voor hoe actief landen hun milieubeleid invullen. Deze score wordt jaarlijks gepubliceerd en kijkt onder andere naar klimaatmaatregelen, landbouwimpact en duurzaamheid van het landgebruik. Door deze scores te koppelen aan uitstootcijfers uit de landbouwsector (in verhouding tot de hoeveelheid geproduceerd voedsel), kunnen we zien of beleid echt effect heeft.

In onderstaande kaart combineren we beide datasets: de kleur laat zien hoe goed een land scoort op milieubeleid, terwijl de grootte van de cirkel aangeeft hoeveel CO₂-uitstoot er vrijkomt per ton voedsel dat wordt geproduceerd.

Een deel van de verschillen in uitstoot per ton voedsel tussen landen lijkt te verklaren door verschillen in milieubeleid. Landen met een hoge EPI-score (een maat voor hoe goed het milieubeleid is) hebben op deze kaart vaak kleine blauwe bubbels. Dat betekent dat ze relatief weinig CO₂ uitstoten per ton geproduceerd voedsel.

Wat opvalt, is dat landen met veel uitstoot meestal ook een lage EPI-score hebben. Dat patroon lijkt erop te wijzen dat landen met minder streng milieubeleid gemiddeld minder duurzaam of efficiënt produceren. Tegelijkertijd is het belangrijk om te bedenken dat ook andere dingen invloed hebben, zoals technologie, geld en de manier waarop de landbouw in een land is georganiseerd.

Toch laat de kaart duidelijk zien dat landen met strenger beleid, zoals Zwitserland of Noorwegen, niet alleen goed scoren op milieugebied, maar ook hun landbouwuitstoot laag houden. Dat ondersteunt het idee dat milieubeleid echt verschil kan maken in hoe duurzaam voedsel wordt geproduceerd.

Ook al bewijst dit nog niet dat beleid de oorzaak is, de visualisatie is wel een sterk beginpunt om verder te onderzoeken hoe beleid en milieueffecten met elkaar samenhangen.

### Argument 2: Landen die duurzaamheid belonen (via subsidies) in landbouwsubsidies stoten minder uit (per hectare)

Niet alleen milieuregels zijn belangrijk, maar ook geld speelt een grote rol. Overheden sturen landbouw niet alleen met wetten en regels, maar ook met subsidies. In sommige landen zijn die subsidies gekoppeld aan duurzaamheid: boeren krijgen bijvoorbeeld alleen steun als ze milieuvriendelijk werken, hun CO₂-uitstoot beperken of bijdragen aan biodiversiteit. In andere landen vloeit het meeste subsidiegeld juist nog steeds naar vervuilende vormen van landbouw, zoals intensieve veehouderij.

In deze visualisatie vergelijken we per land het aandeel duurzame en niet-duurzame subsidies. Daarnaast kijken we naar hoeveel CO₂ er wordt uitgestoten per hectare landbouwgrond. Op die manier maken we zichtbaar dat landen die hun subsidies richten op duurzaamheid, gemiddeld ook schoner produceren.

In [2]:
import pandas as pd
import plotly.graph_objects as go

# Data inladen
emissies = pd.read_csv("../data1/FAOSTAT_landbouw_emissies_2022.csv")
productie = pd.read_csv("../data1/FAOSTAT_landbouw_productie_2022.csv")
epi = pd.read_csv("../data1/epi2022results05302022.csv")

# Emissies per land (in kiloton)
emissie_per_land = emissies.groupby("Area")["Value"].sum().reset_index()
emissie_per_land.columns = ["Country", "Emissions_kt"]

# Productie per land (in ton omzetten naar kiloton)
productie_per_land = productie.groupby("Area")["Value"].sum().reset_index()
productie_per_land.columns = ["Country", "Production_ton"]
productie_per_land["Production_kt"] = productie_per_land["Production_ton"] / 1000

# Combineren en berekenen uitstoot per ton
combined = pd.merge(emissie_per_land, productie_per_land, on="Country")
combined["Emissions_per_ton"] = combined["Emissions_kt"] / combined["Production_kt"]

# EPI-score koppelen
epi = epi.rename(columns={"country": "Country", "EPI.new": "EPI_Score"})
final = pd.merge(combined, epi[["Country", "EPI_Score"]], on="Country")

# 🔍 Filter landen met te weinig landbouwproductie (minder dan 100 kiloton)
final = final[(final["Production_kt"] > 5000) & (final["Emissions_kt"] > 50)]


# Percentielen voor kleurcontrasten
p10 = final["Emissions_per_ton"].quantile(0.10)
p90 = final["Emissions_per_ton"].quantile(0.90)

# EPI als achtergrondkaart
choropleth = go.Choropleth(
    locations=final["Country"],
    locationmode="country names",
    z=final["EPI_Score"],
    colorscale="YlGn",
    zmin=0,
    zmax=100,
    showscale=True,
    colorbar=dict(
        title="EPI Score",
        x=1.02,
        len=0.6
    )
)

# Scatter: uitstoot per ton landbouwproductie
scatter = go.Scattergeo(
    locationmode='country names',
    locations=final["Country"],
    text=final["Country"],
    marker=dict(
        size=4,
        color=final["Emissions_per_ton"],
        colorscale=["#ffffcc", "#ffcc66", "#ff9933", "#cc3300", "#660000"],
        cmin=p10,
        cmax=p90,
        colorbar=dict(
            title="CO₂-uitstoot per ton landbouwproductie",
            tickangle=0,
            x=0.5,
            xanchor="center",
            y=-0.12,
            len=1.0,
            thickness=20,
            tickvals=[
                round(p10, 3),
                round((p10 + p90) / 2, 3),
                round(p90, 3)
            ]
        )
    ),
    name="Uitstoot per ton"
)

# Layout
fig = go.Figure(data=[choropleth, scatter])
fig.update_layout(
    title="CO₂-uitstoot per ton landbouwproductie en milieubeleid (EPI-score) per land (2022)",
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='equirectangular'
    ),
    height=550
)

fig.show()
